In [1]:
import pickle
import numpy as np
import pandas as pd
import itertools
from mastml.mastml import Mastml
from mastml.feature_generators import ElementalFeatureGenerator, ElementalFractionGenerator
# from mastml.feature_generators import ElementalFeatureGenerator


matminer is an optional dependency. To install matminer, do pip install matminer


In [2]:

pfile_X = "Codes/bandgap_pbe_X.pickle"
pfile_Y = "Codes/bandgap_pbe_Y.pickle"

with open(pfile_X, 'rb') as f:
    data = pickle.load(f)

In [12]:
# for k,v in data.items():
    # print(k)

# n = 74992
# acc = 0
# for id in data['icsd_id']:
#     if not id:
#         acc += 1
# print(data)
# print(acc / n)
# print(list(numpy.unique(data['reference'])))
# print(list(numpy.unique(data['comments'])))
# print(list(numpy.unique(data['bandgap type'])))
# print(list(numpy.unique(data['comp method'])))

x_clean = data[['composition', 'structure', 'space group']]
# print("x_clean", x_clean)

composition = x_clean['composition']
structure = x_clean['structure']

m = float("inf")

# foo = structure[m]
# compositions = [composition[i] for i in range(m)]
compositions = [comp for i, comp in enumerate(composition) if i < m]

# sites = foo['sites']
# print("compositions", compositions)
print("num compositions", len(compositions))

# l = []
# for s in sites:
#     l.append(s['species'])

# element_names = ['H', 'He', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Ne', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'Ar', 'K',
#                  'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr', 'Rb',
#                  'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 'Sn', 'Sb', 'Te', 'I', 'Xe', 'Cs',
#                  'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta',
#                  'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi', 'Po', 'At', 'Rn', 'Fr', 'Ra', 'Ac', 'Th', 'Pa',
#                  'U', 'Np', 'Pu', 'Am', 'Cm', 'Bk', 'Cf', 'Es', 'Fm', 'Md', 'No', 'Lr', 'Rf', 'Db', 'Sg', 'Bh', 'Hs', 'Mt',
#                  'Ds', 'Rg', 'Cn', 'Nh', 'Fl', 'Mc', 'Lv', 'Ts', 'Og']

# print(bar)
# decomposeComposition(bar)
# print(bar)
# EnsembleModelFeatureSelector()
# generate_elementfraction_features(bar)

num compositions 74992


In [13]:
# df = pd.DataFrame([compositions])
df = pd.DataFrame(compositions)
# print("df", df)
# ! USE ElementFraction instead ElementalFractionGenerator
# e = ElementalFeatureGenerator(df)
e = ElementalFractionGenerator(df)
e.fit()
x, y = e.transform()
print("vector shape", x.shape, "y", y)


vector shape (74992, 118) y None


In [15]:
# list(x.items())
# print(*x.items(), sep="\n")
# print(x.iloc[0])

## verify rows sum to 1
# total = x.sum(axis=1, numeric_only= True)
# print("total", total)

material_group_char = ["O", "B", "F", "Cl", "Br", "I"]
for mat in material_group_char:
    print("Grouping for", mat)
    st1 = 100
    st2 = 10
    # thresholds = [i/st1 for i in range(0,st1)] + [i/st2 for i in range(1,st2)]
    thresholds = [0]
    for th in thresholds:
        # print("threshold", th)
        O_group = x[x[mat] > th]
        not_O_group = x[x[mat] <= th]
        # print("o group shape for theshold", th, O_group.shape, O_group.shape[0]/x.shape[0])
        perc_in_group = O_group.shape[0]/x.shape[0]
        print(f"\ttheshold:\t{th}\n\tpercent in {mat}_group:\t{perc_in_group}")
        if perc_in_group == 0:
            continue
        # print("not o group shape for theshold", th, not_O_group.shape, not_O_group.shape[0]/x.shape[0])

        # th = 0.5
        O_group = x[x[mat] > th]
        not_O_group = x[x[mat] <= th]

        o_centroid = O_group.mean(axis=0)
        not_o_centroid = not_O_group.mean(axis=0)
        x_cent = x.mean(axis=0)

        # groups = (o_centroid, not_o_centroid, x_cent)
        # group_names = ("o_centroid", "not_o_centroid", "x_cent")
        groups = {f"{mat}_centroid": o_centroid, f"not_{mat}_centroid": not_o_centroid, "x_centroid": x_cent}

        pairs = list(itertools.combinations(list(groups.items()), 2))
        # print(*pairs, sep="\n")

        distances = []
        for ((g1_name, g1), (g2_name, g2)) in pairs:
            dist = np.sqrt(np.sum([(a-b)*(a-b) for a, b in zip(g1, g2)]))
            distances.append(((g1_name, g2_name), dist))
        for (n1, n2), d in distances:
            print(f"\t\tdist {n1} {n2}\t{d}")

    # print("o_centroid", o_centroid)
    # print("not_o_centroid", not_o_centroid)
    # print("x_cent", x_cent)


Grouping for O
	theshold:	0
	percent in O_group:	0.4545018135267762
		dist O_centroid not_O_centroid	0.54322672127658
		dist O_centroid x_centroid	0.29632919130016977
		dist not_O_centroid x_centroid	0.24689752997641023
Grouping for B
	theshold:	0
	percent in B_group:	0.04636494559419672
		dist B_centroid not_B_centroid	0.2953864981862926
		dist B_centroid x_centroid	0.2816909192686254
		dist not_B_centroid x_centroid	0.013695578917667163
Grouping for F
	theshold:	0
	percent in F_group:	0.0855824621292938
		dist F_centroid not_F_centroid	0.5070204229979047
		dist F_centroid x_centroid	0.46362836684790687
		dist not_F_centroid x_centroid	0.0433920561499979
Grouping for Cl
	theshold:	0
	percent in Cl_group:	0.04140441647109025
		dist Cl_centroid not_Cl_centroid	0.40092635377725583
		dist Cl_centroid x_centroid	0.3843262320512259
		dist not_Cl_centroid x_centroid	0.016600121726030037
Grouping for Br
	theshold:	0
	percent in Br_group:	0.019255387241305738
		dist Br_centroid not_Br_centroid

In [6]:
## Compute distances